In [16]:
# Import libraries and utils
%run '../../data_processing/api_calls/utils.ipynb'

In [4]:
# Load dataframe and convert into tuple
nl_simlex = pd.read_csv("../../data/nl-simlex-999.txt", delimiter='\t')
tuples_list = list(zip(nl_simlex['word1'], nl_simlex['word2']))

In [5]:
# Set client
load_dotenv()
client = OpenAI()

In [20]:
# Define the prompt
prompt = ("Beoordeel de semantische gelijkenis van elk woordpaar op een schaal van 0 tot 10, "
          "waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. "
          "Gebruik twee decimalen. Jouw antwoord moet strikt voldoen aan de structuur: "
          "[('woord1', 'woord2', <score>), ('woord3', 'woord4', <score>), ...]. Geef geen extra uitleg of context."
          "\n\n---\n\n"
          "[('oud', 'nieuw'), ('slim', 'intelligent'), ('hard', 'moeilijk'),], "
          "[('slecht', 'vreselijk'), ('moeilijk', 'gemakkelijk'), ('slim', 'dom'),], "
          "[('gelukkig', 'vrolijk'), ('hard', 'stoer'), ('gelukkig', 'blij'),],]"
          )

In [21]:
print(prompt)

Beoordeel de semantische gelijkenis van elk woordpaar op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. Gebruik twee decimalen. Jouw antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>), ('woord3', 'woord4', <score>), ...]. Geef geen extra uitleg of context.

---

[('oud', 'nieuw'), ('slim', 'intelligent'), ('hard', 'moeilijk'),], [('slecht', 'vreselijk'), ('moeilijk', 'gemakkelijk'), ('slim', 'dom'),], [('gelukkig', 'vrolijk'), ('hard', 'stoer'), ('gelukkig', 'blij'),],]


In [8]:
# Define model
model = "gpt-3.5-turbo-0125"

In [13]:
messages = [{"role": "user", "content": prompt}] # Create message according to OpenAI API docs

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)


print(completion.choices[0].message.content)

[('oud', 'nieuw', 0.20), ('slim', 'intelligent', 0.90), ('hard', 'moeilijk', 0.10)]


In [15]:
completion.choices[0].message.content

"[('oud', 'nieuw', 0.20), ('slim', 'intelligent', 0.90), ('hard', 'moeilijk', 0.10)]"

In [18]:
# Extract content
content = completion.choices[0].message.content

# Convert to list of tuples
list_of_tuples = ast.literal_eval(content)
list_of_tuples

[('oud', 'nieuw', 0.2),
 ('slim', 'intelligent', 0.9),
 ('hard', 'moeilijk', 0.1)]

In [26]:
completion

ChatCompletion(id='chatcmpl-9IkYmCbcwo1tqYKxULGKJCVNwnVzp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="[('oud', 'nieuw', 0.20), ('slim', 'intelligent', 0.90), ('hard', 'moeilijk', 0.10)]", role='assistant', function_call=None, tool_calls=None))], created=1714255908, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=40, prompt_tokens=147, total_tokens=187))

In [19]:
# content = parse_content(completion.choices[0].message.content)
# content

In [21]:
# Initialize an empty Pandas DataFrame
df = pd.DataFrame(columns=['word1', 'word2', 'similarity_score'])

# Append to Pandas dataframe
for row in content:
    df = df.append({'word1': row[0], 'word2': row[1], 'similarity_score': row[2]}, ignore_index=True)

# Show results
df

,word1,word2,similarity_score
0,oud,nieuw,1.0
1,slim,intelligent,0.9
2,hard,moeilijk,0.2


In [24]:
test_list = "[('oud', 'nieuw', 4.50), ('slim', 'intelligent', 9.50), ('hard', 'moeilijk', 2.00), ('gelukkig', 'vrolijk', 7.50), ('hard', 'stoer', 6.50), ('snel', 'razendsnel', 9.50), ('gelukkig', 'blij', 8.50), ('kort', 'lang', 1.50), ('dom', 'stom', 8.50), ('vreemd', 'eigenaardig', 9.50), ('breed', 'smal', 2.00), ('slecht', 'vreselijk', 7.50), ('makkelijk', 'moeilijk', 6.00), ('moeilijk', 'gemakkelijk', 6.50), ('slim', 'dom', 8.00), ('krankzinnig', 'gek', 9.00), ('gelukkig', 'kwaad', 2.00), ('uitgebreid', 'groot', 5.00), ('moeilijk', 'simpel', 7.50), ('nieuw', 'vers', 6.50), ('scherp', 'saai', 1.50), ('vlug', 'snel', 8.50), ('dom', 'dwaas', 8.50), ('prachtig', 'fantastisch', 8.50), ('eigenaardig', 'vreemd', 9.50), ('gelukkig', 'boos', 3.00), ('smal', 'breed', 2.00), ('eenvoudig', 'gemakkelijk', 9.50), ('oud', 'vers', 5.50), ('kennelijk', 'duidelijk', 8.00), ('betaalbaar', 'goedkoop', 9.00), ('leuk', 'grootmoedig', 2.00), ('raar', 'vreemd', 9.50), ('vreemd', 'normaal', 4.50), ('slecht', 'immoreel', 9.00), ('verdrietig', 'grappig', 1.50), ('prachtig', 'geweldig', 8.50), ('schuldig', 'beschaamd', 9.50), ('mooi', 'prachtig', 8.50), ('zelfverzekerd', 'zeker', 8.00), ('dom', 'onderontwikkeld', 7.00), ('groot', 'flexibel', 3.50), ('aardig', 'wreed', 3.50), ('ongeduldig', 'nerveus', 6.50), ('groot', 'breed', 6.50), ('sterk', 'trots', 7.50), ('onnodig', 'noodzakelijk', 5.00), ('onrustig', 'jong', 4.00), ('dom', 'intelligent', 8.00), ('slecht', 'geweldig', 1.50), ('moeilijk', 'eenvoudig', 8.00), ('noodzakelijk', 'belangrijk', 7.50), ('slecht', 'verschrikkelijk', 8.00), ('boos', 'blij', 2.50), ('eerlijk', 'schuldig', 1.50), ('makkelijk', 'zwaar', 1.50), ('makkelijk', 'flexibel', 8.50), ('stellig', 'zeker', 9.50), ('essentieel', 'noodzakelijk', 9.00), ('anders', 'normaal', 7.50), ('sluw', 'slim', 8.50), ('cruciaal', 'belangrijk', 8.50), ('hard', 'wreed', 2.00), ('kinderachtig', 'dwaas', 8.00), ('zelden', 'zeldzaam', 9.50), ('vriendelijk', 'vrijgevig', 8.00), ('breekbaar', 'kil', 1.50), ('lang', 'smal', 1.50), ('groot', 'zwaar', 7.50), ('ruw', 'koel', 4.50), ('bizar', 'vreemd', 9.00), ('illegaal', 'immoreel', 9.50), ('slecht', 'schuldig', 2.50), ('modern', 'oud', 2.50), ('nieuw', 'oud', 3.00), ('saai', 'grappig', 3.50), ('gelukkig', 'jong', 6.00), ('makkelijk', 'groot', 3.00), ('geweldig', 'vreselijk', 7.50), ('klein', 'enorm', 1.50), ('beleefd', 'gepast', 8.50), ('bescheiden', 'beschaamd', 7.50), ('exotisch', 'zeldzaam', 6.50), ('dom', 'slim', 8.00), ('heerlijk', 'prachtig', 8.00), ('opvallend', 'duidelijk', 6.50), ('angstig', 'bang', 8.00), ('formeel', 'juist', 8.00), ('somber', 'saai', 5.00), ('heerlijk', 'vrolijk', 7.50), ('ongelukkig', 'kwaad', 3.00), ('verdrietig', 'vreselijk', 7.50), ('ziek', 'gek', 9.00), ('gewelddadig', 'boos', 7.50), ('beladen', 'zwaar', 6.00), ('vuil', 'goedkoop', 1.50), ('elastisch', 'flexibel', 9.00), ('hard', 'dicht', 4.00), ('onlangs', 'nieuw', 6.50),]"

In [27]:
word_pairs = ast.literal_eval(test_list)
word_pairs

[('oud', 'nieuw', 4.5),
 ('slim', 'intelligent', 9.5),
 ('hard', 'moeilijk', 2.0),
 ('gelukkig', 'vrolijk', 7.5),
 ('hard', 'stoer', 6.5),
 ('snel', 'razendsnel', 9.5),
 ('gelukkig', 'blij', 8.5),
 ('kort', 'lang', 1.5),
 ('dom', 'stom', 8.5),
 ('vreemd', 'eigenaardig', 9.5),
 ('breed', 'smal', 2.0),
 ('slecht', 'vreselijk', 7.5),
 ('makkelijk', 'moeilijk', 6.0),
 ('moeilijk', 'gemakkelijk', 6.5),
 ('slim', 'dom', 8.0),
 ('krankzinnig', 'gek', 9.0),
 ('gelukkig', 'kwaad', 2.0),
 ('uitgebreid', 'groot', 5.0),
 ('moeilijk', 'simpel', 7.5),
 ('nieuw', 'vers', 6.5),
 ('scherp', 'saai', 1.5),
 ('vlug', 'snel', 8.5),
 ('dom', 'dwaas', 8.5),
 ('prachtig', 'fantastisch', 8.5),
 ('eigenaardig', 'vreemd', 9.5),
 ('gelukkig', 'boos', 3.0),
 ('smal', 'breed', 2.0),
 ('eenvoudig', 'gemakkelijk', 9.5),
 ('oud', 'vers', 5.5),
 ('kennelijk', 'duidelijk', 8.0),
 ('betaalbaar', 'goedkoop', 9.0),
 ('leuk', 'grootmoedig', 2.0),
 ('raar', 'vreemd', 9.5),
 ('vreemd', 'normaal', 4.5),
 ('slecht', 'immoreel',

In [36]:
word_dict = {}

for w1, w2, score in word_pairs:
    if (w1, w2) not in word_dict:
        dict[(w1, w2)] = {'word1': w1, 'word2': w2}
    dict[(w1, w2)][f'similarity_score_{sample_index+1}'] = score

TypeError: 'type' object does not support item assignment

In [34]:
# Converting dictionary to pandas DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index')
df

,word1,word2,similarity_score
0,oud,nieuw,4.5
1,slim,intelligent,9.5
2,hard,moeilijk,2.0
3,gelukkig,vrolijk,7.5
4,hard,stoer,6.5
...,...,...,...
94,beladen,zwaar,6.0
95,vuil,goedkoop,1.5
96,elastisch,flexibel,9.0
97,hard,dicht,4.0


In [32]:
# df = pd.DataFrame.from_dict(word_dict, orient='index')
# df.reset_index(drop=True, inplace=True)
# df